In [1]:
import os
import getpass

import numpy as np
import splusdata 
import pandas as pd
from astropy.table import Table
import astropy.units as u

import context

In [2]:
login = "jessica"
password = getpass.getpass()

········


In [3]:
conn = splusdata.connect(login, password)

In [4]:
My_Query = f"""SELECT det.ID, det.ra, det.dec, sgq.PROB_STAR
               FROM dr2.detection_image as det
               JOIN dr2_vacs.star_galaxy_quasar as sgq ON (sgq.ID = det.ID)
               WHERE sgq.PROB_STAR>0.8"""

In [5]:
Result = conn.query(My_Query)

finished


In [6]:
Result[:5]

ID,RA,DEC,PROB_STAR
bytes24,float64,float64,float64
DR2.SPLUS-s02s01.067560,324.9769963676468,-0.8284107871826184,1.0
DR2.SPLUS-s02s01.034029,324.92221866255494,-1.3781302021139012,1.0
DR2.SPLUS-s02s01.034884,324.780309808846,-1.3605479533955114,0.9
DR2.SPLUS-s02s01.038679,324.0435487994416,-1.2837548124513534,1.0
DR2.SPLUS-s02s01.038779,325.04210595071214,-1.2834814871850828,1.0


In [15]:
tablename = os.path.join('C:/Users/amori/Dropbox/splus-halpha (1)/tables/Literature_new_phot_structural_parameters_8arcsec_class_star.fits')
tab = Table.read(tablename)

aperture = 'aper_6'
ps = 0.55 * u.arcsec / u.pix
size = 256 * u.pix
r = np.sqrt(2) * size * ps # arcsec
r = r.to(u.degree).value

for i, t in enumerate(tab):
    ra0 = t["ALPHA_J2000"]
    dec0 = t["DELTA_J2000"]
table = conn.query(f"""SELECT det.ID, det.ra, det.dec
               FROM dr2.detection_image as det 
               WHERE 1 = CONTAINS( POINT('ICRS', det.ra, det.dec), CIRCLE('ICRS', {ra0}, {dec0}, {r}) )""")

finished


In [17]:
result_table = conn.query(f"""SELECT det.ID, det.ra, det.dec 
                              FROM dr2.detection_image AS det  
                              JOIN dr2_vacs.star_galaxy_quasar as sgq ON (sgq.ID = det.ID)
                              WHERE (sgq.PROB_STAR>0.8) AND 1=CONTAINS( POINT('ICRS', det.ra, det.dec), CIRCLE('ICRS', {ra0}, {dec0}, {r}) )""")

finished


In [18]:
res = conn.get_last_result()

In [19]:
res

ID,RA,DEC
bytes1,float64,float64
